In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


2026-02-18 20:48:30.460806: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# -----------------------------
# Load CIFAR-10
# -----------------------------
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

class_names = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]

y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 476s 3us/step


In [3]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")


In [ ]:
mobilenet_base = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

mobilenet_base.trainable = False  


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 10s 1us/step


In [5]:
mobilenet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(preprocess_input),   
    mobilenet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)  
], name="cifar10_mobilenetv2")

mobilenet_model.summary()


Model: "cifar10_mobilenetv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1),
]

history = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 981s 1s/step - accuracy: 0.6798 - loss: 0.9184 - val_accuracy: 0.8130 - val_loss: 0.5413 - learning_rate: 0.0010
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 1045s 1s/step - accuracy: 0.7442 - loss: 0.7332 - val_accuracy: 0.8204 - val_loss: 0.5231 - learning_rate: 0.0010
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 1119s 2s/step - accuracy: 0.7601 - loss: 0.6911 - val_accuracy: 0.8386 - val_loss: 0.4867 - learning_rate: 0.0010


In [7]:
test_loss_m, test_acc_m = mobilenet_model.evaluate(x_test, y_test, verbose=0)

print("MobileNetV2 (frozen) test accuracy:", test_acc_m)
print("MobileNetV2 (frozen) test loss:", test_loss_m)


MobileNetV2 (frozen) test accuracy: 0.8219000101089478
MobileNetV2 (frozen) test loss: 0.5124844908714294


In [8]:
mobilenet_base.trainable = True

for layer in mobilenet_base.layers[:-30]:
    layer.trainable = False

print("Trainable layers in backbone:",
      sum(l.trainable for l in mobilenet_base.layers),
      "/", len(mobilenet_base.layers))


Trainable layers in backbone: 30 / 154


In [9]:
mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history_ft = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)


Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 1414s 2s/step - accuracy: 0.7172 - loss: 0.8264 - val_accuracy: 0.8244 - val_loss: 0.5178
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 1285s 2s/step - accuracy: 0.7765 - loss: 0.6415 - val_accuracy: 0.8344 - val_loss: 0.4684
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 1341s 2s/step - accuracy: 0.7996 - loss: 0.5759 - val_accuracy: 0.8530 - val_loss: 0.4124


In [10]:
test_loss_m_ft, test_acc_m_ft = mobilenet_model.evaluate(x_test, y_test, verbose=0)

print("MobileNetV2 (fine-tuned) test accuracy:", test_acc_m_ft)
print("MobileNetV2 (fine-tuned) test loss:", test_loss_m_ft)


MobileNetV2 (fine-tuned) test accuracy: 0.8529000282287598
MobileNetV2 (fine-tuned) test loss: 0.4304276406764984
